# Correspondance des organisations avec la base de données SIRENE

In [1]:
import os
os.chdir('/Users/Raphael/INRIA/travail/anHALytics/') #Select your working directory
cwd = os.getcwd()
import codecs
import pandas as pd
import os.path
from langdetect import detect_langs
import numpy as np
import matplotlib.pyplot as plt
spec=['à','á','â','ç','è','é','ê','ë','ì','í','î','ï','ñ','ò','ó','ô','ù','ú','û','ü','ý','ÿ',
      '/','*',',','.',' ','?']

### Récupération des données

#### Tables anHALytics

In [3]:
anHALytics={}
for x in os.listdir(cwd+'/json'):
    anHALytics[x[:-5]]=pd.read_json(cwd+'/json/'+x)

#### Table SIRENE (un subset)

In [29]:
subset_SIRENE=pd.read_json('/Users/Raphael/INRIA/travail/SIRENE/subset_SIRENE.json')

### Fonction de normalisation du texte pour effectuer la correspondance

In [5]:
def normalize(string,spec):
    string=string.lower()
    for x in spec:
        string=string.replace(x,'')
    return(string)

### On normalise les colonnes correspondantes aux noms des entitées:

In [30]:
subset_SIRENE['recherche'] = subset_SIRENE['NOMEN_LONG'].apply(lambda x: normalize(x,spec))

In [7]:
anHALytics['ORGANISATION_NAME']['recherche']=anHALytics['ORGANISATION_NAME']['name'].apply(lambda x: normalize(x,spec))

La dernière étape consiste à joindre les deux tables via la colonne 'recherche':

In [31]:
merged=pd.merge(subset_SIRENE,anHALytics['ORGANISATION_NAME'],on='recherche')

Cependant, il reste quelques problèmes

In [32]:
merged.groupby(['organisationID','name']).SIREN.count().sort_values(ascending=False).head()

organisationID  name                                         
285             Institut National de la Recherche Agronomique    165
859             Bull SAS                                          36
860             Bull SAS                                          36
3899            BRGM                                              35
15529           Alten                                             29
Name: SIREN, dtype: int64

Premièrement, on remarque que des noms sont associés à plusieurs lignes de la base SIRENE.

In [12]:
subset_SIRENE.groupby("SIREN").SIREN.count().sort_values(ascending=False).head()

SIREN
180089013    848
180036048    166
180070039    165
130015506     98
199411117     73
Name: SIREN, dtype: int64

On remarque que dans la table SIRENE, des codes SIREN apparaissent sur plusieurs ligne et ce malgré leur charactère unique. Les code SIREN sont bels et bien uniques, néanmoins chaque code est apparenté à une entreprise, or une entreprise peut possèder plusieurs établissements qui seront donc enregistrés dans la base de donnée. On se sert donc de l'attribut "SIEGE" pour ne laisser qu'une seul ligne pour chaque code SIREN, car une entreprise possède un et un seul siège:

In [33]:
subset_SIRENE=subset_SIRENE[subset_SIRENE['SIEGE']==1]
merged=pd.merge(subset_SIRENE,data['ORGANISATION_NAME'],on='recherche')

In [34]:
merged.groupby(['organisationID','name']).SIREN.count().sort_values(ascending=False).head()

organisationID  name   
9772            ISI        3
18341           STI        3
17568           ISI        3
16765           ODYSSEE    3
18970           LSI        3
Name: SIREN, dtype: int64

Le problème semble persister de manière plus légère, regardons de plus près celui-ci:

In [38]:
merged[merged['organisationID']==9772][['organisationID','name','recherche','SIREN','NOMEN_LONG']]

,organisationID,name,recherche,SIREN,NOMEN_LONG
99,9772,ISI,isi,829080878,I.S.I.
104,9772,ISI,isi,340414390,ISI
109,9772,ISI,isi,447749300,ISI


On remarque sans suprise que les noms d'entreprise de sont pas "uniques", ou du moins les sigles ne le sont pas. Nous sommes donc dans des cas ambigüs, que je ne sais pas régler de manière automatique (pas assez d'information dans les autres champs de la base SIRENE), donc je choisi de ne pas inclure ces lignes dans le résultat final (le temps que je trouve une solution alternative)

In [39]:
orgid=merged.groupby(['organisationID']).SIREN.count()[merged.groupby(['organisationID']).SIREN.count()<2].index.values
org_SIREN=merged[merged['organisationID'].isin(orgid)][['organisationID','SIREN']]
org_SIREN.head()

,organisationID,SIREN
4,2214,821134491
28,2090,821591062
29,13967,821591062
30,19324,821591062
31,19325,821591062


Cette nouvelle table permettra de relier la table organisation et la table SIRENE.